In [1]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib
%pip install accelerate
%env NCCL_P2P_DISABLE="1"
%env NCCL_IB_DISABLE="1"

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-vn7gm8w4
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-vn7gm8w4
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit ecf72c716f6378a9bb2191ed884b5ce9428dcc81
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
env: NCCL_P2P_DISABLE="1"
env: NCCL_IB_DISABLE="1"


In [2]:
from pathlib import Path 
import gc    
import torch
import os
import torch
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%cd /root/mesh-transformer

/root/mesh-transformer


In [4]:
from accelerate import notebook_launcher
def training_function(num_processes=2):
    autoencoder = MeshAutoencoder( 
        checkpoint_quantizer = True,
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,   
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8, 
        attn_decoder_depth  = 4,
        attn_encoder_depth = 2) 
    
    pkg = torch.load("./mesh-autoencoder.ckpt.epoch_0_avg_loss_-0.06490_recon_0.3518_commit_-0.8334.pt")  
    autoencoder.load_state_dict(pkg['model'], strict = False) 
  
    dataset = MeshDataset.load("./mesh-transformer-datasets/objverse_250f_490.7M_all_17561_labels_568425_5_min_x5_aug.npz")  
    dataset2 = MeshDataset.load("./mesh-transformer-datasets/objverse_250f_98.1M_all_17561_labels_113685_5_min_x1_aug.npz")
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./mesh-transformer-datasets/shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
    dataset.data.extend(dataset2.data)  
    dataset2 = MeshDataset.load("./mesh-transformer-datasets/shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
    dataset.data.extend(dataset2.data) 
    dataset.sort_dataset_keys() 
         
    autoencoder.commit_loss_weight = 0.5
    autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                                 batch_size=16,
                                                 grad_accum_every =4,
                                                 learning_rate = 1e-4,
                                                 checkpoint_every_epoch=1)  
    _loss1 = autoencoder_trainer.train(14445,  diplay_graph= False)        
 
args = ()
notebook_launcher(training_function, args, num_processes=1)

Launching training on 4 GPUs.
[MeshDataset] Loaded 568425 entries
[MeshDataset] Created from 568425 entries
[MeshDataset] Loaded 113685 entries
[MeshDataset] Created from 113685 entries
[MeshDataset] Loaded 2156 entries
[MeshDataset] Created from 2156 entries
[MeshDataset] Loaded 21560 entries
[MeshDataset] Created from 21560 entries


W0613 02:28:54.769000 140656738912064 torch/multiprocessing/spawn.py:145] Terminating process 828 via signal SIGTERM
W0613 02:28:54.778000 140656738912064 torch/multiprocessing/spawn.py:145] Terminating process 830 via signal SIGTERM
W0613 02:28:54.782000 140656738912064 torch/multiprocessing/spawn.py:145] Terminating process 831 via signal SIGTERM
E0613 02:28:58.038000 140656738912064 torch/distributed/elastic/multiprocessing/api.py:695] failed (exitcode: 1) local_rank: 1 (pid: 829) of fn: training_function (start_method: fork)
E0613 02:28:58.038000 140656738912064 torch/distributed/elastic/multiprocessing/api.py:695] Traceback (most recent call last):
E0613 02:28:58.038000 140656738912064 torch/distributed/elastic/multiprocessing/api.py:695]   File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/api.py", line 656, in _poll
E0613 02:28:58.038000 140656738912064 torch/distributed/elastic/multiprocessing/api.py:695]     self._pc.join(-1)
E0613 02:28:58

ChildFailedError: 
============================================================
training_function FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2024-06-13_02:28:52
  host      : bc73b43bc8c9
  rank      : 1 (local_rank: 1)
  exitcode  : 1 (pid: 829)
  error_file: /tmp/torchelastic_4yssv9ys/none_kbmz0wrr/attempt_0/1/error.json
  traceback : Traceback (most recent call last):
    File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 347, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_501/1982501885.py", line 27, in training_function
      autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
    File "<@beartype(meshgpt_pytorch.trainer.MeshAutoencoderTrainer.__init__) at 